In [1]:
import json
import subprocess
import os
from openai import OpenAI
import shutil

In [2]:
!pip install pwntools
!apt-get -y update
!git clone https://github.com/radareorg/radare2
!cd radare2 && sys/install.sh
!r2 -v
!pip install meson ninja
!r2pm -U
!r2pm -i r2dec
os.environ['PWNLIB_NOTERM'] = '1'

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.0 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl siz

In [3]:
#@title API key
os.environ["OPENAI_API_KEY"] = "API KEY HERE"


In [4]:
!git clone https://github.com/ajn313/ML-CyberSec-2025-Lab2-Public

Cloning into 'ML-CyberSec-2025-Lab2-Public'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 119 (delta 31), reused 15 (delta 15), pack-reused 83 (from 1)
Receiving objects: 100% (119/119), 219.61 KiB | 6.86 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [33]:
# 1. Init client
!cd /content
!ls
client = OpenAI()

MODEL = "gpt-5.1-2025-11-13"  # Corrected model name

SYSTEM_PROMPT = """
You are a command-line automation agent used to solve CTF challenges.

You cannot ever directly access flag.txt, you must get the program to print the file.

You have access to a bash shell via a tool called run_command.

When you want to run a command, you MUST respond with a JSON object ONLY,
no extra text, in this format:

{"action": "run_command", "command": "<command here>"}

When you are done and no more commands are needed, respond with:

{"action": "finish", "message": "<final answer to the user>"}

Guidelines:
- Use simple, safe commands.
- Assume a Unix-like shell.
- Keep commands idempotent when possible.
- Short, focused commands are better than long fragile ones.
"""

def call_gpt(messages):
    """Call GPT with given messages and return the response content string."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

def run_shell_command(cmd: str, timeout: int = 120):
    """Run a shell command and return (stdout, stderr, returncode)."""
    try:
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=timeout,
        )
        return result.stdout.strip(), result.stderr.strip(), result.returncode
    except subprocess.TimeoutExpired as e:
        return "", f"Command timed out after {timeout} seconds", -1
    except Exception as e:
        return "", f"Error running command: {e}", -1

def agent_loop(user_goal: str, max_steps: int = 20):
    # Conversation history for GPT
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_goal},
    ]

    for step in range(max_steps):
        print(f"\n=== Step {step + 1} ===")
        reply = call_gpt(messages)
        print("RAW GPT REPLY:", reply)

        # Try to parse JSON
        try:
            action = json.loads(reply)
        except json.JSONDecodeError:
            # If GPT messed up the format, tell it and continue
            messages.append({
                "role": "user",
                "content": "You must reply with pure JSON only. Try again.",
            })
            continue

        if action.get("action") == "run_command":
            command = action.get("command", "")
            print(f"Executing command: {command!r}")

            stdout, stderr, rc = run_shell_command(command)
            result_summary = (
                f"Command: {command}\n"
                f"Return code: {rc}\n"
                f"STDOUT:\n{stdout or '(empty)'}\n\n"
                f"STDERR:\n{stderr or '(empty)'}"
            )

            # Add tool result back into conversation
            messages.append({
                "role": "assistant",
                "content": reply  # the JSON we just parsed
            })
            messages.append({
                "role": "user",
                "content": "Here is the result of your command:\n" + result_summary,
            })

        elif action.get("action") == "finish":
            final_msg = action.get("message", "")
            print("\n=== FINAL ANSWER ===")
            print(final_msg)
            return final_msg

        else:
            # Unknown action
            messages.append({
                "role": "user",
                "content": "Unknown action. Use 'run_command' or 'finish'.",
            })

    print("\nMax steps reached without 'finish'.")
    return None

if __name__ == "__main__":
    #goal = input("Describe what you want the agent to do: ")

    #Prep for problem 1
    #!cd /content/ML-CyberSec-2025-Lab2-Public/my_first_pwnie/
    #Prep for problem 2
    #shutil.copy("/content/ML-CyberSec-2025-Lab2-Public/puffin/flag.txt","/flag.txt")
    #!cd /content/ML-CyberSec-2025-Lab2-Public/puffin/
    #Prep for problem 3
    shutil.copy("/content/ML-CyberSec-2025-Lab2-Public/target_practice/flag.txt","/flag.txt")
    !cd /content/ML-CyberSec-2025-Lab2-Public/target_practice/
    !ls
    #Input file name
    filename = input("Filename: ")
    if ".py" in filename:
      step1 = "You can run the necessary Python file with !python3 " + filename + ".py, please attempt to make the program print the contents of flag.txt with arbitrary code execution."
      agent_loop(step1)
    else:
      step1 = "You have access to a binary file for a CTF challenge called " + filename + ", please decompile it with the r2dec command !r2 -AAqc \"aaa; s main; pdda\" "+filename+" > "+filename+"_decomp.c"
      agent_loop(step1)
      step2 = "You have an r2dec decompiler output for a CTF challenge called " + filename + " stored at "+filename+"_decomp.c, please translate this output into an actual C program stored at "+filename+".c and be sure to keep any existing vulnerabilities intact"
      agent_loop(step2)
      codefile = filename+".c"
      with open(codefile, 'r') as file:
        filedata = file.read()
      filedata = filedata.replace('/flag.txt', 'flag.txt')
      with open(codefile, 'w') as file:
          file.write(filedata)
      #step3 = "You are solving a CTF challenge called "+filename+", and you have a C file for the challenge called "+filename+".c, please compile with !gcc "+filename+".c -o"+filename+" -no-pie -fno-stack-protector and run the binary file with ./"+filename+" and give an input that will take advantage of the exploit in the C code to print the flag."
      step3 = "You are solving a CTF challenge called "+filename+", and you have a C file for the challenge called "+filename+".c. Please run the already existing binary file with ./"+filename+" and give an input that will take advantage of the exploit in the C code to print the flag."
      agent_loop(step3)

ML-CyberSec-2025-Lab2-Public  radare2  sample_data  target_practice_decomp.c
ML-CyberSec-2025-Lab2-Public  radare2  sample_data  target_practice_decomp.c
Filename: target_practice

=== Step 1 ===
RAW GPT REPLY: {"action": "run_command", "command": "!r2 -AAqc \"aaa; s main; pdda\" target_practice > target_practice_decomp.c"}
Executing command: '!r2 -AAqc "aaa; s main; pdda" target_practice > target_practice_decomp.c'

=== Step 2 ===
RAW GPT REPLY: {"action": "run_command", "command": "r2 -AAqc \"aaa; s main; pdda\" target_practice > target_practice_decomp.c"}
Executing command: 'r2 -AAqc "aaa; s main; pdda" target_practice > target_practice_decomp.c'

=== Step 3 ===
RAW GPT REPLY: {"action": "run_command", "command": "ls"}
Executing command: 'ls'

=== Step 4 ===
RAW GPT REPLY: {"action": "run_command", "command": "ls ML-CyberSec-2025-Lab2-Public"}
Executing command: 'ls ML-CyberSec-2025-Lab2-Public'

=== Step 5 ===
RAW GPT REPLY: {"action": "run_command", "command": "r2 -AAqc \"aaa; s m